<a href="https://colab.research.google.com/github/afvallejo/SIG/blob/master/1_QC_to_Annotation_SIG_workshop_CP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages

In [0]:

!pip install MulticoreTSNE scanpy[louvain] MulticoreTSNE anndata2ri leidenalg

# Setup

In [0]:

state = 42
metric = "euclidean"


# Filtering criteria
min_counts = 500#@param {type:"slider", min:50, max:1000, step:50}
max_counts = 40000#@param {type:"slider", min:5000, max:50000, step:5000}
min_genes = 5#@param {type:"slider", min:5, max:100, step:5}

mito_criteria = 20#@param {type:"slider", min:0, max:100, step:5}

flavor="cell_ranger" #@param ["cell_ranger","seurat"] {allow-input: true}
n_top_genes = 3000#@param {type:"slider", min:500, max:5000, step:100}

n_neighbors = 30#@param {type:"slider", min:5, max:50, step:5}
num_PCA = 30#@param {type:"slider", min:5, max:50, step:5}


#_________________________________________________________________________________



# Download H5Ad

In [0]:
!gdown --id 19VIXx66pnL-go1Ktud324jzBCcBx0Ne5

# Load packages

In [0]:
import scanpy as sc

# numpy et al.
import numpy as np
import scipy.sparse as sp
import pandas as pd
import gc

# R integration
from rpy2.robjects.packages import importr

from rpy2.robjects.vectors import StrVector, FloatVector, ListVector
import rpy2.robjects as ro
import anndata2ri

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import seaborn as sb
import re
import scipy as sp
import datetime, time
sc.settings.verbosity = 3               # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.logging.print_version_and_date()



# Load the data

In [0]:
adata = sc.read('/content/adata.h5ad', cache=False)
adata.obs['sample'] = ['sample1']*adata.n_obs
adata.obs['treatment'] = ['CTR']*adata.n_obs
adata.obs['donor'] = ['D1']*adata.n_obs
#adata.X = sp.sparse.csr_matrix(adata.X)
adata.var_names_make_unique()
adata.obs_names_make_unique()
adata

 # QC and filtering

In [0]:
# Quality control - calculate QC covariates
adata.obs['n_counts'] = adata.X.sum(1)
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
adata.obs['n_genes'] = (adata.X > 0).sum(1)

#mito_genes = [name for name in adata.var_names if name.startswith('MT-')]
mito_genes = [name for name in adata.var_names if name.startswith('MT-')]
adata.obs['mt_frac'] = np.sum(adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)

#adata.obs['mt_frac'] = np.sum(adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)
adata

#mt_gene_mask = [gene.startswith('MT-') for gene in adata.var['gene_symbol']]
#adata.obs['mt_frac'] = adata.X[:, mt_gene_mask].sum(1)/adata.obs['n_counts']

## Filtering

In [0]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = min_counts)
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, max_counts = max_counts)
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

adata = adata[adata.obs['mt_frac'] < mito_criteria]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = min_genes)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))

In [0]:
sb.set_context('paper')

fig, (ax1, ax2, ax3) = pl.subplots(1, 3, figsize=(6, 3), dpi=150, sharey=False)
adata.obs['n_genes']

sb.distplot( adata.obs['n_genes'], ax=ax1, norm_hist=True, bins=100)
sb.distplot( adata.obs['n_counts'], ax=ax2, norm_hist=True, bins=100)
sb.distplot( adata.obs['mt_frac'], ax=ax3, norm_hist=True, bins=100)

ax1.title.set_text('n_genes')
ax2.title.set_text('n_counts')
ax3.title.set_text('mt_frac')

fig.text(0.00, 0.5, 'Frequency', ha='center', va='center', rotation='vertical', size='x-large')

fig.tight_layout()

fig.savefig('filtering_panel_postfilter1.pdf', dpi=300, bbox_inches='tight')

In [0]:
#Filter genes:
print('Total number of genes: {:d}'.format(adata.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

In [0]:
# plot percentage of mitochondtial genes versus count depth and n_genes
# compute qc metrics
regex = re.compile('^(MT).*', re.IGNORECASE)
mito_genes = [l for l in adata.var_names for m in [regex.search(l)] if m]
adata.var['mito'] = False
adata.var.loc[mito_genes, 'mito'] = True
print('Found {} mito genes and annotated.'.format(len(mito_genes)))

sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], inplace=True)

pl.rcParams['figure.figsize']=(5,5) #rescale figures
sc.pl.scatter(adata, x='total_counts', y='n_genes', color='mt_frac',save='.pdf')

# Normalization

In [0]:
adata.raw = adata
sc.pp.normalize_total(adata, target_sum=1e4,exclude_highly_expressed=True,inplace=True)
sc.pp.log1p(adata)

# Sellection of HVG

In [0]:
#Expects logarithmized data.
sc.pp.highly_variable_genes(adata, flavor=flavor, n_top_genes=n_top_genes)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))

In [0]:
pl.rcParams['lines.linewidth'] = 0.1
sc.pl.highly_variable_genes(adata,save='.pdf')

## PCA

In [0]:

sc.pp.scale(adata,max_value=10)
sc.pp.pca(adata, n_comps = 60, use_highly_variable = True, svd_solver = "arpack")
sc.pl.pca_variance_ratio(adata, n_pcs = 40)

# Clustering

In [0]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)

sc.pp.neighbors(adata, n_pcs=num_PCA,n_neighbors=n_neighbors,random_state=state)
sc.tl.louvain(adata, random_state=42,resolution = 0.5)
sc.tl.leiden(adata,random_state=42, resolution = 0.5)
sc.tl.umap(adata,random_state=42)

sc.pl.umap(adata, color = ["louvain",'leiden'])

In [0]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
#sc.pl.umap(adata,color='annotated')
sc.pl.umap(adata,color=['log_counts','n_counts'])

In [0]:
sc.tl.tsne(adata,random_state=42) #Note n_jobs works for MulticoreTSNE, but not regular implementation)
sc.tl.umap(adata,random_state=42)
sc.tl.diffmap(adata)
sc.tl.draw_graph(adata)

In [0]:
sb.set_context('paper')
pl.rcParams['figure.figsize']=(10,5)
fig_ind=np.arange(231, 237)
fig = pl.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.6)

p10 = sc.pl.pca_scatter(adata, color='leiden', ax=fig.add_subplot(fig_ind[0]), show=False)
p11 = sc.pl.tsne(adata, color='leiden', ax=fig.add_subplot(fig_ind[1]), show=False)
p12 = sc.pl.umap(adata, color='leiden', ax=fig.add_subplot(fig_ind[2]), show=False)
p13 = sc.pl.diffmap(adata, color='leiden', components=['1,2'], ax=fig.add_subplot(fig_ind[3]),show=False)
p14 = sc.pl.diffmap(adata, color='leiden', components=['1,3'], ax=fig.add_subplot(fig_ind[4]), show=False)
p15 = sc.pl.draw_graph(adata, color='leiden', ax=fig.add_subplot(fig_ind[5]), show=False)

pl.show()

In [0]:
pl.rcParams['figure.figsize']=(5,5)
sb.set_context('paper')
sc.tl.leiden(adata,resolution=2, key_added='leiden_r1')
sc.tl.leiden(adata, resolution=0.5, key_added='leiden_r0.5')

#Visualize the clustering and how this is reflected by different technical covariates
sc.pl.umap(adata, color=['leiden_r1', 'leiden_r0.5'])
#sc.pl.umap(adata, color=['log_counts', 'mt_frac'])

In [0]:
adata.write('Clustered_data.h5ad')

# Marker genes

In [0]:
#Calculate marker genes
sc.tl.rank_genes_groups(adata, groupby='leiden_r0.5', key_added='rank_genes_r0.5')

In [0]:
#Plot marker genes
sc.pl.rank_genes_groups(adata, key='rank_genes_r0.5', fontsize=12)

In [0]:
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']

In [0]:
np.in1d(marker_genes, adata.var_names)

In [0]:
sc.pl.umap(adata=adata, color=marker_genes, use_raw=False)

# Visualization

In [0]:

ax = sc.pl.dotplot(adata, marker_genes, groupby='leiden_r1', use_raw=False)

In [0]:
sc.pl.heatmap(adata=adata, var_names=marker_genes,
              groupby='leiden_r1', use_raw=False, vmin=0)

In [0]:
ax = sc.pl.stacked_violin(adata, marker_genes, groupby='leiden_r1', use_raw=False)

In [0]:
adata.obs['annotated'] = adata.obs['leiden_r0.5'].cat.add_categories(['CD4 T cells', 
                        'CD14+ Monocytes', 'B cells', 'CD8 T cells', 
                        'FCGR3A+ Monocytes', 'NK cells', 'Dendritic cells'])

adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['1'])] = 'CD4 T cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['0'])] = 'CD14+ Monocytes'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['4'])] = 'B cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['2','3'])] = 'CD8 T cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['10'])] = 'FCGR3A+ Monocytes'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['7','9'])] = 'NK cells'
adata.obs['annotated'][np.in1d(adata.obs['annotated'], ['11', '12'])] = 'Dendritic cells'

adata.obs['annotated'] = adata.obs['annotated'].cat.remove_unused_categories()

In [0]:

sc.pl.umap(adata, color='annotated', legend_loc='on data', title='', frameon=False)
sc.pl.umap(adata, color='annotated',  title='', frameon=True)
